# Мэтчинг

## Описание проекта

Заказчику необходимо внедрить систему поиска товаров по запросу пользователя.

Имеются наборы данных уже преобразованных в эмбединги:
- База данных объектов (3 млн строк) - векторы размерностью 72
- Тренировочная выборка (100к строк) - векторы, имеющие лейблы (совпадающие товары из базы выше)
- Тестовая выборка (100к строк) - векторы для проверки работы алгоритма

### Цель проекта

Найти алгоритм, который находит наиболее удачные совпадения из существующей базы.

Метрикой выбрана accuracy@5, то есть алгоритм отработал удачно, если из 5 результатов предоставлен релевантный.

## План работы

Применение алгоритмов:
- Faiss
- Annoy
- Qdrant

Сопоставление результатов

## Решение

### Импорт

In [1]:
import pandas as pd
import numpy as np

import faiss

from sklearn.preprocessing import StandardScaler

import time
import json

### Блок функций

In [2]:
def accuracy_n(target_df, idxs):
    acc = 0
    for target, el in zip(target_df.values.tolist(), idxs.tolist()):
        acc += int(target in [base_index[r] for r in el])

    return(100 * acc / len(idxs))

### Загрузка и осмотр данных

In [3]:
base_df = pd.read_csv('datasets/base.csv',index_col=0)
display(base_df.head())
print(base_df.shape)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


(2918139, 72)


In [4]:
train_df = pd.read_csv('datasets/train.csv',index_col=0)
display(train_df.head())
print(train_df.shape)

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,30.413513,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,140.331070,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,211.607820,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,242.650650,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,103.392670,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


(100000, 73)


In [5]:
validation_df = pd.read_csv('datasets/validation.csv',index_col=0)
display(validation_df.head())
print(validation_df.shape)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,75.215750,...,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,53.128998,...,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,168.920320,...,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,75.206730,...,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,102.086914,...,-123.77025,45.635944,-134.25893,13.735359,70.61763,15.332115,154.56812,101.700640,-1171.892332,-125.307890


(100000, 72)


In [6]:
validation_answer_df = pd.read_csv('datasets/validation_answer.csv',index_col=0)
display(validation_answer_df.head())
print(validation_answer_df.shape)

,Expected
Id,
100000-query,2676668-base
100001-query,91606-base
100002-query,472256-base
100003-query,3168654-base
100004-query,75484-base


(100000, 1)


Данные представляют из себя эмбединги товаров. Нестандартизированные, это нужно исправить. Но ради интереса попробуем и без этого

Данные уже разбиты на трейн и тест, выделяем таргеты

In [7]:
targets_train = train_df["Target"]
train_df.drop("Target", axis=1, inplace=True)
targets_val = validation_answer_df['Expected']

### Faiss 

Составим индекс. Для начала пусть будет 1024 кластеров, по которым будет осуществляться поиск (число велико, но мы в будущем будем обходить несколько кластеров при поиске)

Увеличение числа кластеров дает бо'льшую производительность за счет точности.

Затем подберем этот гиперпараметр

In [8]:
dims = base_df.shape[1]
n_cells = 1024
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

Обучать индекс будем на 50000 векторов. Вероятно этого хватит, но нужно проверить.

Также составляем словарь, где ключи - порядковые номера датафрейма `base_df` , значения - индекс(строка) в `base_df`

In [9]:
base_index = {k: v for k, v in enumerate(base_df.index.to_list())}

In [10]:
idx_l2.train(np.ascontiguousarray(base_df.values[:50000, :]).astype('float32'))
idx_l2.add(np.ascontiguousarray(base_df.values).astype('float32'))

In [11]:
distances, idxs = idx_l2.search(np.ascontiguousarray(train_df.values).astype('float32'), 5)

In [12]:
accuracy_n(targets_train, idxs)

4.244

Accuracy@5 чуть выше 4%. Стандартизируем векторы и запустим алгоритм заново

In [13]:
scaler = StandardScaler()
base_array = scaler.fit_transform(base_df)
train_array = scaler.fit_transform(train_df)
validation_array = scaler.fit_transform(validation_df)

In [14]:
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [15]:
idx_l2.train(np.ascontiguousarray(base_array[:50000, :]).astype('float32'))
idx_l2.add(np.ascontiguousarray(base_array).astype('float32'))

In [16]:
%%time

distances, idxs = idx_l2.search(np.ascontiguousarray(train_array).astype('float32'), 5)
accuracy_n(targets_train, idxs)

CPU times: total: 21.4 s
Wall time: 4.11 s


49.679

Алгоритм отработал за 4 с небольшим секунды. Стандартизация заметно улучшает метрику. Проверим насколько сильно упадет результат на валидации

In [17]:
%%time

distances, idxs = idx_l2.search(np.ascontiguousarray(validation_array).astype('float32'), 5)
accuracy_n(targets_val, idxs)

CPU times: total: 21.1 s
Wall time: 4.12 s


49.333

Метрика на валидации сильно похожа на трейн выборку. Предполагаю, что имеет схожее распределение.

Попробуем увеличить количество кластеров для поиска `nprobe`. Время работы алгоритма при этом должно увеличиться. Предполагаю, что этот гиперпараметр будет иметь схожий эффект с гиперпараметром `n_cells` - перебор бо'льшего числа векторов при увеличении времени работы.

In [18]:
idx_l2.nprobe = 32

In [19]:
%%time

distances, idxs = idx_l2.search(np.ascontiguousarray(train_array).astype('float32'), 5)
accuracy_n(targets_train, idxs)

CPU times: total: 8min 49s
Wall time: 1min 42s


64.697

Больше кластеров - точнее поиск (т.к. правильный результат может быть в соседнем кластере). Время работы увеличивается линейно

Что делаем дальше. 

Проверим следующий алгоритм:

Находим оптимальные гиперпараметры:
- Увеличиваем количество результатов поиска - от 5 до выхода на плато. `accuracy_n` увеличится, но осложнится последующая задача ранжирования.
- Увеличиваем nprobe - тут время будет расти линейно, получаем trade между скоростью работы и точностью алгоритма

Составляем матрицу:
- Каждый вектор из query конкатенируем с вектором из выдачи. То же самое проделываем для валидационной выборки
- В качестве таргета добавляем лейбл 1 или 0. Делаем так для каждого вектора из query, в итоге получаем датасет из 100_000 * idx.shape[1] строк
- Обучаем ml-модели для нахождения 1 на трейне. На выдаче получаем скоры. Следим за переобучением на валидационной выборке.
- Первые 5 скоров и будут результатом выдачи, по которым будет считаться метрика.



Реализуем первый шаг - найдем гиперпараметры для faiss
Пройдемся вложенным циклом и упакуем результаты в словарь. Струкрура словаря
```
metrics = {
    '5 соседей':{
        '1 кластер': [время_5_1, accuracy_train, accuracy_val], 
        '2 кластера': [время_5_2, accuracy_train, accuracy_val],
        ...,
        '128 кластеров': [время_5_2, accuracy_train, accuracy_val]
    },
    '10 соседей':{
        '1 кластер': [время_10_1, accuracy_train, accuracy_val], 
        '2 кластера': [время_10_2, accuracy_train, accuracy_val],
        ...,
        '128 кластеров': [время_10_2, accuracy_train, accuracy_val]
}
```

In [22]:
metrics = {}
c = time.time()
for neighbours in range(5,101,5): #ищем от 5 до 100 соседей
    metrics[neighbours] = {}
    for nprobe in [2**x for x in range(5)]: #ищем в разном числе кластеров
        a = time.time()
        idx_l2.nprobe = nprobe
        distances_train, idxs_train = idx_l2.search(np.ascontiguousarray(train_array).astype('float32'), neighbours)
        for i,v in enumerate(idxs_train):
            if -1 in v:
                idxs_train[i] = np.array([idx if idx != -1 else 0 for idx in v])
        
        distances_val, idxs_val = idx_l2.search(np.ascontiguousarray(validation_array).astype('float32'), neighbours)
        for i,v in enumerate(idxs_val):
            if -1 in v:
                idxs_val[i] = np.array([idx if idx != -1 else 0 for idx in v])
            
        b = time.time()
        print(f'\rОбработан запрос: neighbours = {neighbours}, nprobe = {nprobe}, общее время выполнения {b-c:.2f} секунд', end='')
        metrics[neighbours][nprobe] = [f'Затраченное время = {(b-a)/2:.2f} секунд', accuracy_n(targets_train, idxs_train), accuracy_n(targets_val, idxs_val)]

Обработан запрос: neighbours = 100, nprobe = 16, общее время выполнения 4765.58 секунд

In [25]:
with open('datasets/metrics_faiss.json', "w", encoding="utf-8") as file:
    json.dump(metrics, file)

In [29]:
metrics = json.load(open('datasets/metrics_faiss.json'))
metrics

{'5': {'1': ['Затраченное время = 4.19 секунд', 49.679, 49.333],
  '2': ['Затраченное время = 8.19 секунд', 57.153, 56.757],
  '4': ['Затраченное время = 15.18 секунд', 60.919, 60.591],
  '8': ['Затраченное время = 28.74 секунд', 62.886, 62.609],
  '16': ['Затраченное время = 54.31 секунд', 64.074, 63.761]},
 '10': {'1': ['Затраченное время = 4.46 секунд', 51.916, 51.555],
  '2': ['Затраченное время = 8.17 секунд', 59.914, 59.602],
  '4': ['Затраченное время = 15.03 секунд', 64.06, 63.854],
  '8': ['Затраченное время = 27.58 секунд', 66.272, 66.122],
  '16': ['Затраченное время = 53.50 секунд', 67.631, 67.468]},
 '15': {'1': ['Затраченное время = 4.42 секунд', 52.955, 52.541],
  '2': ['Затраченное время = 8.06 секунд', 61.233, 60.922],
  '4': ['Затраченное время = 15.18 секунд', 65.58, 65.389],
  '8': ['Затраченное время = 27.98 секунд', 67.917, 67.797],
  '16': ['Затраченное время = 54.11 секунд', 69.358, 69.227]},
 '20': {'1': ['Затраченное время = 4.74 секунд', 53.606, 53.218],
  '2